In [ ]:
import pandas as pd

In [ ]:
file=pd.read_excel("/content/Onyx Data -DataDNA Dataset Challenge - Electronics Product Data Ratings - February 2024.xlsx", header=0, sheet_name=1)

In [ ]:
file.head(5)
file.drop(columns=["keys", "id", "dateAdded", "dateUpdated", "imageURLs", "upc", "Unnamed: 24", "reviews.dateSeen"], inplace=True)

In [ ]:
#checking null values in each column
file.isna().sum()

brand                     0
categories                0
colors                 1669
dimension              1209
manufacturer           2667
manufacturerNumber        0
name                      0
reviews.date             61
reviews.doRecommend    1391
reviews.numHelpful     1486
reviews.rating          164
reviews.sourceURLs        0
reviews.text              5
reviews.title             4
reviews.username          0
sourceURLs                0
weight                    0
dtype: int64

In [ ]:
#reviewing inconsistencies in the column colors
file["colors"].unique()

array(['Black', 'Multicolor', nan,
       'Blue,Graphite,Blue steel,Black,Merlot,Blizzard',
       'Multicolor,Black,Grey',
       'Siriusxm Sxezr1h1 Xm,Siriusxm Sxezr1v1 Xm', 'Blue',
       'Red,Pink,Yellow,Blue,Bordeaux Pink,Cinnabar Red,Black,Viridian Blue,Charcoal Black',
       'Navy,Black,Gray,Blue', 'Grey', 'BlackGrey', 'Black,Gray,Red,Blue',
       'White', 'Gray,Color', 'Black,White', 'Cream',
       'Black,Silver,BlackRed,SilverBlack', 'Black Ash'], dtype=object)

In [ ]:
#Changing the values to null, since its not a color
pd.set_option('display.max_colwidth', None)
file.loc[file["colors"]=="Siriusxm Sxezr1h1 Xm,Siriusxm Sxezr1v1 Xm", "colors"]=np.nan

In [ ]:
#Reviewing the colors of each products, replacing those that specify the color in the name of the product
colorsgroup=file[["name", "colors"]].reset_index().groupby(by=["name"])[["colors", "index"]].count()

In [ ]:
#filling colors that dont have a color assigned with the color mentioned in the name of the product
black=["Acoustimass 6 Series V Home Theater Speaker System (Black)",
       'Alpine - 6-1/2" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black',
       'JBL - 6" x 8" 2-Way Coaxial Car Speakers with Polypropylene Cones (Pair) - Black',
       "Kicker 41IK5BT2V2 Amphitheater High-Performance Audio System with Bluetooth, Black",
       "Lenovo - AC Adapter for Select Lenovo Yoga Laptops - Black",
       "Lenovo - AC Adapter for Select Lenovo Yoga Laptops - Black",
       "Round LCD/Projector Ceiling Plate - Black"]
green_black=["DreamWave - Tremor Portable Bluetooth Speaker - Green,Black"]

for i in black:
    file.loc[file["name"]==i, "colors"]="Black"
for i in green_black:
    file.loc[file["name"]==i, "colors"]="Green,Black"
file.loc[file["name"]=="BlackGrey", "colors"]="Green,Black"

In [ ]:
#Normalize dimension column
dims = file["dimension"].values
for i in range(len(dims)):
    s = str(dims[i])
    if s == "3 5/8 in 2 3/4 in 7 5/8 in": s = "3.63 x 2.75 x 7.63"
    if "inches" in s: s = s.replace(" inches", "")
    elif "in" in s: s = s.replace(" in ", "")
    elif "  " in s: s = s.replace("  ", " ")
    dims[i]=s
file["dimension"]=pd.DataFrame(dims)

In [ ]:
#Replace the null values in reviews.text with the reviews.title (for later sentiment analysis based on that column) and reeplace
#empty values in manufacturer column with the number of the manufacter since there isn't a name
file['manufacturer'].fillna(file['manufacturerNumber'], inplace=True)
file["reviews.text"].fillna(file['reviews.title'], inplace=True)

In [ ]:
#Turn the date column for it to only contain year, month and date
from datetime import datetime
ymd=file['reviews.date'].values
for d in range(len(ymd)):
  if pd.notna(ymd[d]):
    dt=str(ymd[d]).split("T")[0]
    dt=datetime.strptime(dt, "%Y-%m-%d")
    ymd[d]=dt
file["reviews.date"]=pd.DataFrame(ymd)

In [ ]:
 #Dropping duplicates
 file.drop_duplicates(inplace=True)
 file.drop_duplicates(subset=['reviews.text', "reviews.date"], inplace=True)

In [ ]:
#Save the changes made to an excel file
file.to_excel("onyx processed.xlsx", index=False)

In [ ]:
#In excel there was seen that in the reviews.text column, in some cases, started with "By" and the username. In that way they weren't considered
#duplicates until it was seen in excel, where once comparing it with another comment given by the same user it was clear that they were duplicates.
#They were manually removed using filters.